## Install the dependencies

In [3]:
!pip install transformers
!pip install accelerate # Designed to facilitate training deep learning models across different hardware.
!pip install bitsandbytes # Transformers library that helps with the quantization of the model.
!pip install datasets
!pip install trl # Training transformer models with reinforcement learning and supervised fine-tuning.
!pip install peft # Parametric efficient fine-tuning of large language models for downstream tasks.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.


In [1]:
from google.colab import userdata
import os
hf_token = userdata.get('HF_TOKEN')
if not hf_token:
    raise ValueError("Secret 'HF_TOKEN' not found. Please add it via the Secrets pane.")

# Optionally set it as env var so Hugging Face libraries detect it automatically
os.environ['HF_TOKEN'] = hf_token

#### Quantization
Load the pretrained model with quantization

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Apply the 8-bit quantization to the model using the BitesAndBytesConfig class
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quantized_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config = bnb_config,
                    device_map = "auto")

tokenizer = AutoTokenizer.from_pretrained(model_name)
input = tokenizer("Natalia sold clips to 48 of her friends in April, and then she sold half as \
many clips in May. How many clips did Natalia sell altogether in April and May?", return_tensors="pt").to('cuda')

response = quantized_model.generate(**input, max_new_tokens = 100)
print(tokenizer.batch_decode(response, skip_special_tokens=True))

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? 48 / 2 = 24. 48 + 24 = 72.\nWhat is the total number of clips Natalia sold altogether in April and May?\n## Step 1: Calculate the number of clips sold in May.\nNatalia sold half as many clips in May as she did in April, so we need to divide the number of clips sold in April by 2. 48 / 2 = 24.\n\n## Step 2: Calculate the total number of clips']


## Training the model

### Preprocess the dataset

In [3]:
from datasets import load_dataset

# Preprocess the dataset
dataset = "openai/gsm8k"
data = load_dataset(dataset, 'main')

# pad token to be the eos_token of the tokenizer. The eos_token is a special token that represents the end of a sentence.
tokenizer.pad_token = tokenizer.eos_token
data = data.map(lambda samples: tokenizer(samples["question"], samples["answer"], truncation=True, padding="max_length", max_length=100), batched=True)
train_sample = data["train"].select(range(400))

display(train_sample)

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 400
})

In [4]:
print(train_sample[:1])

{'question': ['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'], 'answer': ['Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'], 'input_ids': [[128000, 45, 4306, 689, 6216, 27203, 311, 220, 2166, 315, 1077, 4885, 304, 5936, 11, 323, 1243, 1364, 6216, 4376, 439, 1690, 27203, 304, 3297, 13, 2650, 1690, 27203, 1550, 42701, 689, 4662, 31155, 304, 5936, 323, 3297, 30, 128000, 45, 4306, 689, 6216, 220, 2166, 14, 17, 284, 1134, 2166, 14, 17, 28, 1187, 2511, 1187, 27203, 304, 3297, 627, 45, 4306, 689, 6216, 220, 2166, 10, 1187, 284, 1134, 2166, 10, 1187, 28, 5332, 2511, 5332, 27203, 31155, 304, 5936, 323, 3297, 627, 827, 220, 5332, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

### LoRA configurations

In [5]:
import peft # parametric efficient fine-tuning.
from peft import LoraConfig # class LoraConfig for implementing LoRA.

lora_config = LoraConfig(
    r=16, # rank of matrix
    lora_alpha=16, # set alpha equal to rank
    target_modules=["q_proj", "v_proj"], #target_modules are the names of the modules we want to train.
    lora_dropout=0.1,
    bias="none", # bias is a bias type to be updated during training. Its value can be none, all, or lora_only.
    # Setting its value to all or lora_only, will update the corresponding biases during training. We set its value to none.

    task_type="CAUSAL_LM"
)

### Set the training arguments

In [6]:
from transformers import TrainingArguments
import os

working_dir = './'
output_directory = os.path.join(working_dir, "lora")

training_args = TrainingArguments(
    output_dir = output_directory, # Save the model’s checkpoints and training logs.
    auto_find_batch_size = True, # Automatically find the correct batch size according to the data.
    learning_rate = 3e-4, # To balance the learning speed and stability.
    num_train_epochs=5
)

## Set the trainer

In [8]:
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model = quantized_model,
    args = training_args,
    train_dataset = train_sample,
    peft_config = lora_config,
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False) #for creating batches from the set of training data.
)

trainer.train()

Truncating train dataset:   0%|          | 0/400 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: avineetkumar4 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
500,1.262500
1000,0.858400


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=1000, training_loss=1.0604884338378906, metrics={'train_runtime': 1753.4462, 'train_samples_per_second': 1.141, 'train_steps_per_second': 0.57, 'total_flos': 9014088499200000.0, 'train_loss': 1.0604884338378906})

In [9]:
# Save the model.
model_path = os.path.join(output_directory, f"lora_model")
trainer.model.save_pretrained(model_path)

### Load the fine-tuned model

In [1]:
model_path = "lora/lora_model"

from peft import AutoPeftModelForCausalLM
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

loaded_model = AutoPeftModelForCausalLM.from_pretrained(
                                        model_path,
                                        quantization_config = bnb_config,
                                        device_map = 'auto')

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
input = tokenizer("Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?", return_tensors="pt").to('cuda')

response = loaded_model.generate(**input, max_new_tokens = 100)
print(tokenizer.batch_decode(response, skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####\n#### 72 ####']
